In [1]:
import os

In [2]:
%pwd

'e:\\disease_classification_ck\\research'

In [3]:
os.chdir('e:/disease_classification_ck')

In [4]:
%pwd

'e:\\disease_classification_ck'

Update entity now
as we have updated config.yaml, no secrets we have, params.yaml also, we have nothing to update.
entity is return type of the function.like Configbox, int
If you dont have inbuilt return type, you can define custom return type like below:

In [5]:
#config_entity.py under entity
# data ingestion related config/entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Update configuration manager inside src

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
             root_dir=config.root_dir,
             source_URL=config.source_URL,
             local_data_file=config.local_data_file,
             unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
#as DataIngestionConfig is the custom return type as entity.

Update the components

In [7]:
import os
import urllib.request as request #to download the dataset from the url
import zipfile #to unzip the dataset
from src.cnnClassifier import logger #our custom logger
from src.cnnClassifier.utils.common import get_size #get the size of the dataset

In [8]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    #to download data from the url
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info:\n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    #to unzip the downloaded file
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-07-15 12:42:39,551: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-15 12:42:39,551: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-15 12:42:39,551: INFO: common: Directory created at :artifacts]
[2024-07-15 12:42:39,551: INFO: common: Directory created at :artifacts/data_ingestion]
[2024-07-15 12:42:41,251: INFO: 4232414380: artifacts/data_ingestion/data.zip download! with the following info:
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5486:1C7263:28753C:33ADA4:66950A99
Accept-Ranges: bytes
Date: Mon, 15 Jul 2024 11:42:40 GMT
Via: 1.1 varnish
X-Served-By: cache-lon4226-LON
X-Cache

In [10]:
from pathlib import Path

# Use a raw string to avoid escape sequence issues
path = Path('params.yaml')
path1 = Path('config\config.yaml')
# Example usage
print(path)

params.yaml
